In [27]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# 1- download dataset


In [28]:
def download_mnist_data():
    train_data = datasets.MNIST(
        root = "data",
        download=True,
        train=True,
        transform = ToTensor()
    )

    validation_data = datasets.MNIST(
        root = "data",
        download=True,
        train=False,
        transform = ToTensor()
    )
    
    return train_data, validation_data


In [29]:
# download MNIST dataset
train_data, _ = download_mnist_data()

print("MNIST_data+download")


MNIST_data+download


# 2- create data loader


In [30]:
BATCH_SIZE = 128

train_data_loader = DataLoader(train_data, batch_size=BATCH_SIZE)

# 3- build model


In [31]:
class FeedForwardNet(nn.Module):

    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.dense_layer = nn.Sequential(
            nn.Linear(28*28, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, input_data):
        flattened_data = self.flatten(input_data)
        logits = self.dense_layer(flattened_data)
        predictions = self.softmax(logits)

        return predictions 


In [32]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(f"using {device} device")

feed_forward_net = FeedForwardNet().to(device)

using cuda device


# 4- train


In [33]:
def train_one_epoch(model, data_loader, loss_fn, optimizer, device):
    for inputs, targets in data_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        # calculate loss
        predictions = model(inputs)
        loss = loss_fn(predictions, targets)

        # backpropagate loss and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Loss: {loss.item()}")
        
    

In [34]:
def train(model, data_loader, loss_fn, optimizer, device, epochs):
    for i in range(epochs):
        print(f"Epoch {i+1}")

        train_one_epoch(model, data_loader, loss_fn, optimizer, device)

        print("---------------")


    print("Training is done.")

In [35]:
LEARNING_RATE = 0.001

loss_fn = nn.CrossEntropyLoss()
optimzer = torch.optim.Adam(feed_forward_net.parameters(),
                            lr = LEARNING_RATE)

In [36]:
EPOCHS = 10

train(feed_forward_net, train_data_loader, loss_fn, optimzer, device, EPOCHS)



Epoch 1
Loss: 1.5132404565811157
---------------
Epoch 2
Loss: 1.4961293935775757
---------------
Epoch 3
Loss: 1.492934226989746
---------------
Epoch 4
Loss: 1.4792674779891968
---------------
Epoch 5
Loss: 1.4755655527114868
---------------
Epoch 6
Loss: 1.4733136892318726
---------------
Epoch 7
Loss: 1.4727977514266968
---------------
Epoch 8
Loss: 1.4723819494247437
---------------
Epoch 9
Loss: 1.4730738401412964
---------------
Epoch 10
Loss: 1.4732708930969238
---------------
Training is done.


# 5- save trained model

In [37]:
torch.save(feed_forward_net.state_dict(), "feedforwardnet.pth")

print("Model trained and storeed at feedforwardnet.pth")

Model trained and storeed at feedforwardnet.pth
